# Exemple d'analyse d'un dataset : prix des maisons à Seattle

**Exemple de régressions**

## Importations des librairies courantes

In [ ]:
# Directive pour afficher les graphiques dans Jupyter (inutile si on utilise Spyder)
%matplotlib inline

In [ ]:
# Pandas : librairie de manipulation de données
# NumPy : librairie de calcul scientifique
# MatPlotLib : librairie de visualisation et graphiques
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score

## Lecture du fichier

On dispose des statistiques de vente des maisons du disctrict de Seattle 

<img src="https://sightline-wpengine.netdna-ssl.com/wp-content/uploads/2012/04/Sprawl_Walkability_CS06m_hi.jpg">

- id
- date: Date house was sold(String)
- price: Price of the sold house
- bedrooms: Numer of Bedrooms
- bathrooms: Numer of bathrooms
- sqft_living: Square footage of the living room
- sqrt_log: Square footage of the log
- floors: Total floors in the house
- waterfront: Whether the house has a view a waterfront(1: yes, 0: not)
- view: unknown
- condition: Condition of the house
- grade: unknown
- sqft_above: Square footage of house apart from basement
- sqft_basement: Square footage of the basement
- yr_built: Built year
- yr_renovated: Year when the house was renovated
- zipcode: zipcode of the house
- lat: Latitude coordinate
- long Longitude coordinate
- sqft_living15: Living room area in 2015(implies some renovations)
- sqrt_lot15: Lot area in 2015(implies some renovations)

In [ ]:
#lecture du dataset
df = pd.read_csv("../input/housesalesprediction/kc_house_data.csv")

In [ ]:
df.head(10)

In [ ]:
df.info()

In [ ]:
df.plot(kind="scatter", x="long", y="lat", c="price", cmap="rainbow", s=3, figsize=(12,12))

**Exercice** : visualiser les maisons par quartiers (zipcode)  
**Exercice** : visualiser les maisons de moins de 1 million de dollars par prix. On pourra aussi faire varier la taille du point en fonction de la surface de la propriété
**Exercice** : visualiser la densité des propriétés avec *jointplot*

In [ ]:
df.info()

In [ ]:
df.count()

In [ ]:
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month

On affiche un graphique montrant l'évolution du prix moyen en fonction de la date :

In [ ]:
df.groupby(['year','month'])['price'].mean().plot(kind = 'bar', figsize=(12,8))

et le nombre de maisons vendues :

In [ ]:
df.groupby(['year','month'])['price'].count().plot(kind = 'bar', figsize=(12,8))

On supprime du dataset l'identifiant id, et la date (que nous avons convertie en mois et année)

In [ ]:
df = df.drop(['id','date'], axis=1)

## Recherche de corrélations

On a vu assez clairement la corrélation entre le nombre de pièces et la valeur.  
On va utiliser la fonction *corr* pour calculer systématiquement le degré de corrélation entre deux paramètres :

In [ ]:
tabcorr = df.corr()     # on peut utiliser aussi bos.corr(method='pearson') par exemple

Pour visualiser l'ensemble du tableau de corrélations, on utilise une "carte de températures" (*heatmap*) :

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(abs(tabcorr), cmap="coolwarm")

On peut aussi regrouper les paramètres par *clusters* classés par proximité :

In [ ]:
sns.clustermap(abs(tabcorr), cmap="coolwarm")

On peut tracer uniquement le **dendrogramme** des corrélations entre les caractéristiques. On utilise une forme "condensée" de la matrice de corrélation. *linkage* permet de calculer les distances entre caractéristiques à partir des corrélations

In [ ]:
from scipy.cluster import hierarchy as hc

corr = 1 - df.corr()
corr_condensed = hc.distance.squareform(corr)
link = hc.linkage(corr_condensed, method='ward')
plt.figure(figsize=(12,12))
den = hc.dendrogram(link, labels=df.columns, orientation='left', leaf_font_size=10)

On s'intéresse plus précisément à la valeur des maisons :

In [ ]:
correlations = tabcorr.price
print(correlations)

On élimine la ligne MV elle-même (qui est forcément à 1) :

In [ ]:
correlations = correlations.drop(['price'],axis=0)

Les corrélations fortement négatives sont aussi significatives que les positives ; on considère donc les valeurs absolues, et on trie par ordre décroissant :

In [ ]:
print(abs(correlations).sort_values(ascending=False))

On voit que la plus forte corrélation concerne la surface

# Régression linéaire multiple

On sépare les caractéristiques continues et discrètes :

In [ ]:
continuous_features = ['sqft_living','sqrt_log','sqft_above','sqft_basement','sqft_living15','sqrt_lot15','lat','long']
discrete_features = ['month','year','bedrooms','bathrooms','floors','waterfront','view','condition',
                     'grade','yr_built','yr_renovated','zipcode']

Pour la régression linéaire, on se limite aux maisons de moins de 1M$, et on élimine les *features* discrètes :

In [ ]:
df1 = df[df.price<1000000].drop(discrete_features, axis=1)

In [ ]:
X = df1.drop(['price'], axis=1)
y = df1.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

On utilise la fonction de régression linéaire multiple de *sklearn* :

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()
lm.fit(X_train, y_train)            # apprentissage
y_pred = lm.predict(X_test)         # prédiction sur l'ensemble de test

On trace le nuage de points pour comparer la prédiction et les résultats attendus :

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_pred)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

ou on peut visualiser la distribution de l'erreur avec *seaborn* :

In [ ]:
sns.distplot(y_test-y_pred)

On peut calculer l'erreur sur les moindres carrés :

In [ ]:
print(np.sqrt(mean_squared_error(y_test, y_pred)))

Ou le *score R2* (rapport des variances estimée/réelle) :
(https://fr.wikipedia.org/wiki/Coefficient_de_d%C3%A9termination)

In [ ]:
scoreR2 = r2_score(y_test, y_pred)
print(scoreR2)

qu'on peut écrire plus simplement :

In [ ]:
lm.score(X_test,y_test)

## Régression par forêts aléatoires

On va utiliser les forêts aléatoires pour la régression. On conserve l'ensemble des caractéristiques

In [ ]:
X = df1.drop(['price'], axis=1)
y = df1.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
from sklearn import ensemble
rf = ensemble.RandomForestRegressor()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)
print(rf.score(X_test,y_test))

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_rf)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

In [ ]:
sns.distplot(y_test-y_rf)

In [ ]:
print(np.sqrt(mean_squared_error(y_test, y_rf)))

**Exercice** : donner l'importance des caractéristiques

In [ ]:
import xgboost as XGB
xgb  = XGB.XGBRegressor()
xgb.fit(X_train, y_train)
y_xgb = xgb.predict(X_test)
print(xgb.score(X_test,y_test))

plt.figure(figsize=(12,12))
plt.scatter(y_test, y_xgb)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

## Exercice : tester la méthode *Gradient Boosting Regressor*

In [ ]:
X = df1.drop(['price'], axis=1)
y = df1.price
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
from sklearn import ensemble
rf = ensemble.GradientBoostingRegressor()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)
print(rf.score(X_test,y_test))

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_rf)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

In [ ]:
sns.distplot(y_test-y_rf)

## Exercice : utiliser le dataset de la valeur des maisons à Boston
https://www.kaggle.com/prasadperera/the-boston-housing-dataset

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.
from pandas import read_csv

In [ ]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df = pd.read_csv('../input/boston-house-prices/housing.csv', header=None, delimiter=r"\s+", names=column_names)
print(df.head(5))

In [ ]:
df.head()

In [ ]:
print(np.shape(df))

In [ ]:
print(df.describe())

In [ ]:
X = df.drop(['MEDV'], axis=1)
y = df.MEDV
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)

In [ ]:
from sklearn import ensemble
rf = ensemble.GradientBoostingRegressor()
rf.fit(X_train, y_train)
y_rf = rf.predict(X_test)
print(rf.score(X_test,y_test))

In [ ]:
plt.figure(figsize=(12,12))
plt.scatter(y_test, y_rf)
plt.plot([y_test.min(),y_test.max()],[y_test.min(),y_test.max()], color='red', linewidth=3)
plt.xlabel("Prix")
plt.ylabel("Prediction de prix")
plt.title("Prix reels vs predictions")

In [ ]:
sns.distplot(y_test-y_rf)